# this notebook shows backtestlib.optimizer.BenchMark etc.

In [457]:
import sys
sys.path.insert(0, "..")
from  backtestlib.optimizer import BenchMark, SampleBased, SpectralCut, SpectralSelection
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [458]:
signal = pd.Series(data = [1,2,2,1.5,-.2],index = ['AAPL','GOOGLE','MMM','BAC','GS'])
signal

AAPL      1.0
GOOGLE    2.0
MMM       2.0
BAC       1.5
GS       -0.2
dtype: float64

In [459]:
c = 0.02
sample_var = np.diag(np.ones(len(signal)))*(1-c) + c*np.ones((len(signal),len(signal)))
sample_var += np.random.normal(scale = 1e-2,size = sample_var.shape)
sample_var = .5*(sample_var + sample_var.T) 
sample_var

array([[0.99836044, 0.02513827, 0.02501334, 0.02922495, 0.02358582],
       [0.02513827, 0.99719232, 0.01271878, 0.02299442, 0.02249128],
       [0.02501334, 0.01271878, 1.01431075, 0.00773109, 0.02301905],
       [0.02922495, 0.02299442, 0.00773109, 1.00264873, 0.02367452],
       [0.02358582, 0.02249128, 0.02301905, 0.02367452, 1.01409687]])

In [460]:
opt = BenchMark()
opt.optimize(signal = signal)

INFO:root:BenchMark initialized


array([0.2, 0.2, 0.2, 0.2, 0.2])

In [461]:
opt = SampleBased()
opt.optimize(signal = signal,sample_var = sample_var)

INFO:root:SampleBased initialized


array([ 0.14993645,  0.33285767,  0.33094079,  0.24429267, -0.05802758])

In [462]:
opt = SpectralCut(delta = 0.4)
opt.optimize(signal = signal,sample_var = sample_var)

INFO:root:SpectralCut initialized with  delta 0.2
INFO:root:Optimal K is 5
INFO:root: return compent [-2.64394582 -0.30107198 -1.64199937 -0.62981829 -1.05644364]


array([ 0.14993645,  0.33285767,  0.33094079,  0.24429267, -0.05802758])

In [463]:
opt = SpectralSelection(delta = 0.2, c = 10)
opt.optimize(signal = signal,sample_var = sample_var)

INFO:root:SpectralCut initialized with  delta 0.2, c 10
INFO:root:Optimal gamma is 0.275
INFO:root: return compent [-2.53057344 -0.03822841 -1.32936763 -0.28148704 -0.66838865]


array([ 0.19528393,  0.28024525,  0.27273908,  0.2630813 , -0.01134955])